# 要約 
以下は、与えられたJupyter Notebookの要約です。

このノートブックは、Kaggleの「20の質問」ゲームをプレイするためのAIエージェントを構築することを目的としています。主に、動的に生成される質問と推測を通じてターゲットワードを当てることを目指し、Gemmaモデルを活用しています。

### 問題
ノートブックは、「20の質問」ゲームにおいて、AIエージェントがユーザーに対して質問をし、回答を元にターゲットワードを推測するプロセスを自動化し、効率的に行うためのシステムを構築するという課題に取り組んでいます。

### 手法およびライブラリ
1. **環境セットアップ**:
   - `immutabledict`、`sentencepiece`、`kaggle_environments`といった必要なライブラリをインストール。
   - GitHubから`gemma_pytorch`リポジトリをクローンし、必要なファイルを作業ディレクトリに移動します。

2. **Gemmaモデルの使用**:
   - Gemmaモデルは、因果言語モデルとして使用され、その設定やトークナイザーをインポート。
   - モデルの重みをロードし、ターンごとに質問および回答を処理するための構造を設計しています。

3. **フォーマッタークラスの定義**:
   - `GemmaFormatter`クラスにより、ユーザーとモデルのターンを管理し、システムプロンプトや少数ショットの例をもとにインタラクティブなセッションを形成します。

4. **質問・推測の解析**:
   - `_parse_response`や`_parse_keyword`といった関数を定義し、AIが生成したレスポンスから質問や推測を適切に抽出し処理しています。

5. **モデルの生成および実行**:
   - 実際にいくつかのGemmaモデル（"2b"、"2b-it"、"7b-it-quant"など）をロードし、生成したプロンプトに基づいてレスポンスを取得。
   - モデルによる生成結果を解析し、最終結果を表示するための処理を行なっています。

このノートブックは、効率的な情報収集や推測によって「20の質問」ゲームの戦略を実行するためのAIエージェントの設計と実装に役立てられています。

---


# 用語概説 
以下は、提示されたJupyter Notebookに関連する専門用語の簡単な解説です。初心者にとって馴染みの薄いものや、このノートブック特有のドメイン知識に焦点を当てています。

### 専門用語の解説

1. **immutabledict**
   - 変更できない辞書型のデータ構造を提供するPythonのパッケージです。一度作成した内容を変更できないため、データの保護やハッシュ可能な値としての利用に役立ちます。

2. **sentencepiece**
   - 自然言語処理で用いられるサブワードトークナイザーです。特にニューラルネットワークモデルの入力データ準備に使われ、トークン化の際に語彙サイズを減少させ、より柔軟なモデルの学習を可能にします。

3. **kaggle_environments**
   - KaggleのAI環境をシミュレートするためのライブラリで、エージェント間の競争を含むさまざまな環境を構築する助けになります。このゲームでは、エージェントが相互に作用し、対戦形式で実行されます。

4. **contextlib**
   - Pythonの標準ライブラリの一部で、コンテキストマネージャーを使用するためのユーティリティが含まれています。`with`ステートメントとともに使用され、リソース管理（開放やクローズ）を簡素化します。

5. **Causal LM (Causal Language Model)**
   - 因果関係を持った言語モデルで、次に予測される単語の確率を前の単語のみに基づいて計算するモデルです。一般に、生成タスクに使用され、テキスト生成や対話システムに適しています。

6. **Tokenization**
   - テキストを小さな単位（トークン）に分割するプロセスです。これにより、モデルが理解し処理しやすい形式になります。トークンは通常、単語、サブワード、さらには文字レベルかもしれません。

7. **温度 (Temperature)**
   - 確率的サンプリングのパラメータであり、出力の多様性を制御します。低い温度はより決定的な出力を生成し、高い温度はランダム性を増やし、さまざまな出力を引き出します。

8. **Top-k Sampling**
   - モデルが予測したトークンの中から上位k個の選択肢を考慮して、ランダムに選択するサンプリング手法です。これにより、出力結果の多様性が向上します。

9. **Top-p Sampling (Nucleus Sampling)**
   - 予測された確率分布から、累積確率がpを超える最小のトークンセットを選び、その中からランダムに選ぶ手法です。より自然なテキスト生成が可能です。

10. **Quantization (量子化)**
    - モデルのパラメータを小さなデータ型に変換するプロセスです。これにより、メモリの使用量が減り、推論速度が向上します。特にデバイスの制約がある場合に有効です。

11. **Iterable (イテラブル)**
    - 繰り返し処理が可能なオブジェクトのことです。リストやタプルのようなコレクションや、カスタムオブジェクトを含みます。Pythonでは`__iter__`メソッドを持つオブジェクトはイテラブルと見なされます。

12. **Context Manager (コンテキストマネージャ)**
    - リソースを管理するための構文で、リソース（ファイル、ネットワークコネクションなど）の開放を自動的に行います。`with`文を使用して、リソースの利用範囲を明確に定義できます。

以上の用語は、Jupyter Notebookの理解を深めるために役立つはずです。特に、機械学習や深層学習においては、これらの概念をしっかり把握することが重要です。

---


In [ ]:
# 環境をセットアップする
# immutabledict と sentencepiece パッケージをインストールします
!pip install -q -U immutabledict sentencepiece 
# kaggle_environmentsの必要なバージョンをインストールします
!pip install -q 'kaggle_environments>=1.14.8'
# GitHubからgemma_pytorchリポジトリをクローンします
!git clone https://github.com/google/gemma_pytorch.git
# 作業用のgemmaディレクトリを作成します
!mkdir /kaggle/working/gemma/
# クローンしたgemma_pytorchからgemmaフォルダ内にファイルを移動します
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

In [ ]:
# 必要なモジュールをインポートします
import sys 
# gemma_pytorchのパスをシステムパスに追加します
sys.path.append("/kaggle/working/gemma_pytorch/") 
import contextlib
import os
import torch
import re
import kaggle_environments
import itertools
# Gemmaの設定をインポートします
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
# Gemmaモデルをインポートします
from gemma.model import GemmaForCausalLM
# トークナイザーをインポートします
from gemma.tokenizer import Tokenizer
# Iterable型をインポートします
from typing import Iterable

In [ ]:
class GemmaFormatter:
    # ターンの開始と終了を示すトークン
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        # システムプロンプトと少数ショットの例をセットします
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        # ユーザーとモデルのターンのフォーマットを定義します
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()  # 状態をリセットします

    def __repr__(self):
        # 現在の状態を返します
        return self._state

    def user(self, prompt):
        # ユーザーのプロンプトを状態に追加します
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        # モデルのプロンプトを状態に追加します
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        # ユーザーのターンを開始します
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        # モデルのターンを開始します
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        # 現在のターンを終了します
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        # 状態を初期化します
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)  # システムプロンプトがあれば追加します
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')  # 少数ショットの例を適用します
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        # ターンを適用し、どのエージェントから始まるかによってフォーマッタを決定します
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)  # フォーマッタを循環させます
        for fmt, turn in zip(formatters, turns):
            fmt(turn)  # フォーマッタを使用してプロンプトを追加します
        return self

In [ ]:
def load_model(VARIANT, device):
    # ウェイトファイルのパスを設定します
    WEIGHTS_PATH = f'/kaggle/input/gemma/pytorch/{VARIANT}/2' 

    @contextlib.contextmanager
    def _set_default_tensor_type(dtype: torch.dtype):
        """デフォルトのtorchデータ型を指定されたdtypeに設定します。"""
        torch.set_default_dtype(dtype)  # デフォルト型を設定
        yield  # 処理を進める
        torch.set_default_dtype(torch.float)  # デフォルト型をfloatに戻す

    # モデルの設定を定義します。
    model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()  # バリアントに応じた設定を取得
    model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")  # トークナイザーのパスを設定
    model_config.quant = "quant" in VARIANT  # 量子化の設定

    # モデルのインスタンス化
    with _set_default_tensor_type(model_config.get_dtype()):  # 指定されたdtypeに設定した状態でモデルを生成
        model = GemmaForCausalLM(model_config)  # Gemmaモデルを初期化
        ckpt_path = os.path.join(WEIGHTS_PATH, f'gemma-{VARIANT}.ckpt')  # チェックポイントのパスを設定
        model.load_weights(ckpt_path)  # ウェイトをロード
        model = model.to(device).eval()  # 指定されたデバイスにモデルを移動し、評価モードに設定
        
    return model  # モデルを返す

In [ ]:
def _parse_keyword(response: str):
    # レスポンスからキーワードを抽出します
    match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)  # **に囲まれた部分を検索
    if match is None:
        keyword = ''  # マッチがなければ空の文字列を設定
    else:
        keyword = match.group().lower()  # マッチした部分を小文字に変換
    return keyword  # 抽出したキーワードを返す

In [ ]:
def _parse_response(response: str, obs: dict):
    # レスポンスと観測データに基づいて質問または推測を解析します
    if obs['turnType'] == 'ask':  # ターンの種類が'ask'の場合
        # レスポンスから最初の質問文を抽出します
        match = re.search(".+?\?", response.replace('*', ''))  # アスタリスクを削除して質問を検索
        if match is None:
            question = "Is it a person?"  # 質問が見つからない場合のデフォルト質問
        else:
            question = match.group()  # マッチした質問文を取得
        return question  # 質問を返す
    elif obs['turnType'] == 'guess':  # ターンの種類が'guess'の場合
        guess = _parse_keyword(response)  # キーワードを解析して取得
        return guess  # 推測を返す
    else:
        # 未知のターンタイプの場合はエラーを発生させます
        raise ValueError("Unknown turn type:", obs['turnType'])

In [ ]:
def interleave_unequal(x, y):
    # 2つのイテラブルxとyを交互に組み合わせます
    return [
        item for pair in itertools.zip_longest(x, y)  # xとyを長さの異なる部分も考慮してペアにする
        for item in pair if item is not None  # Noneでないアイテムのみを抽出
    ]  # 交互に組み合わせたリストを返す

In [ ]:
# システムプロンプトを設定します
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."

# 少数ショットの例を定義します
few_shot_examples = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",  # ゲームの開始を促すメッセージ
    "Is it a thing?", "**no**",  # 質問とその回答
    "Is it a country?", "**yes**",  # 質問とその回答
    "Is it Europe?", "**yes** Now guess the keyword.",  # 質問に対する回答と推測への移行
    "**France**", "Correct!",  # 推測とその結果
]

# GemmaFormatterのインスタンスを作成します
formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

In [ ]:
# 観測データを定義します
obs = {
    'turnType': 'ask',  # 現在のターンの種類は'ask'（質問）であることを示します
    'questions': [  # 質問のリスト
        'Is it a living entity?',  # 質問1
        'Is it man-made?',  # 質問2
        'Can it be held in a single hand?'  # 質問3
    ],
    'answers': [  # 回答のリスト
        'no',  # 質問1に対する回答
        'yes',  # 質問2に対する回答
        'yes'  # 質問3に対する回答
    ]
}

In [ ]:
# フォーマッターをリセットします
formatter.reset()
# ゲームの開始をユーザーに伝えます
formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")
# 質問と回答を交互に組み合わせます
turns = interleave_unequal(obs['questions'], obs['answers'])
# ターンを適用します。モデルから開始することを指定します
formatter.apply_turns(turns, start_agent='model')
# ターンの種類に応じて、適切なプロンプトを追加します
if obs['turnType'] == 'ask':
    formatter.user("Please ask a yes-or-no question.")  # 質問を促すメッセージ
elif obs['turnType'] == 'guess':
    formatter.user("Now guess the keyword. Surround your guess with double asterisks.")  # キーワードの推測を促します
# モデルのターンを開始します
formatter.start_model_turn()

In [ ]:
# フォーマッターの状態を文字列に変換します
prompt = str(formatter)
prompt  # 生成したプロンプトを表示します

In [ ]:
# 使用するマシンタイプを設定します（GPUを指定）
MACHINE_TYPE = "cuda" 
# 指定したマシンタイプに基づいてデバイスを作成します
device = torch.device(MACHINE_TYPE)
# 生成するテキストの最大トークン数を設定します
max_new_tokens = 32
# サンプリングのパラメータを設定します
sampler_kwargs = {
    'temperature': 0.01,  # 出力の温度（ランダムさを制御）
    'top_p': 0.1,  # 確率的サンプリングのパラメータ
    'top_k': 1,  # 上位k個のトークンから選択する
}

# Gemma 2b V2

In [ ]:
# モデルをロードします
model = load_model("2b", device)

# プロンプトに基づいてレスポンスを生成します
response = model.generate(
    prompt,  # 入力プロンプト
    device=device,  # 使用するデバイス
    output_len=max_new_tokens,  # 出力するトークンの最大数
    **sampler_kwargs  # サンプリングのパラメータを展開して渡します
)
# 生成されたレスポンスを解析します
response = _parse_response(response, obs)
# 解析したレスポンスを出力します
print(response)

# Gemma 2b-it V2

In [ ]:
# "2b-it"モデルをロードします
model = load_model("2b-it", device)

# プロンプトに基づいてレスポンスを生成します
response = model.generate(
    prompt,  # 入力プロンプト
    device=device,  # 使用するデバイス
    output_len=max_new_tokens,  # 出力するトークンの最大数
    **sampler_kwargs  # サンプリングのパラメータを展開して渡します
)
# 生成されたレスポンスを解析します
response = _parse_response(response, obs)
# 解析したレスポンスを出力します
print(response)

# Gemma 7b-it-quant V2

In [ ]:
# "7b-it-quant"モデルをロードします
model = load_model("7b-it-quant", device)

# プロンプトに基づいてレスポンスを生成します
response = model.generate(
    prompt,  # 入力プロンプト
    device=device,  # 使用するデバイス
    output_len=max_new_tokens,  # 出力するトークンの最大数
    **sampler_kwargs  # サンプリングのパラメータを展開して渡します
)
# 生成されたレスポンスを解析します
response = _parse_response(response, obs)
# 解析したレスポンスを出力します
print(response)

---

# コメント 

> ## dedq
> 
> コードが明確で、動作が速いです。Kaggleのメンバーに最高の仕事を共有してくれてありがとう！
> 
> 

---